In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(r_non_zeros, p, q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(p, q.T)
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    r_non_zeros_ind = r_non_zeros[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(r_non_zeros_ind, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [2]:
def matrix_factorization(r, k, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = r.shape
    np.random.seed(1)
    
    p = np.random.normal(scale=1./k, size=(num_users, k))
    q = np.random.normal(scale=1./k, size=(num_items, k))
    non_zeros = [(i, j, r[i, j]) for i in range(num_users) for j in range(num_items) if r[i, j] > 0]
    
    r_non_zeros = r.copy()
    
    for step in range(steps):
        
        for i, j, r_val in non_zeros:  # 수정된 부분: r_val로 변경
            eij = r_non_zeros[i, j] - np.dot(p[i, :], q[j, :].T)
            p[i, :] = p[i, :] + learning_rate * (eij * q[j, :] - r_lambda * p[i, :])
            q[j, :] = q[j, :] + learning_rate * (eij * p[i, :] - r_lambda * q[j, :])
        
        rmse = get_rmse(r_non_zeros, p, q, non_zeros)  # 수정된 부분: r_non_zeros 전달
        
        if (step % 10) == 0:
            print('step:', step, 'rmse:', rmse)
    
    return p, q


In [2]:
import pandas as pd

movies = pd.read_csv('../머신러닝/data/ml-latest-small/movies.csv')
ratings = pd.read_csv('../머신러닝/data/ml-latest-small/ratings.csv')

ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

rating_movies = pd.merge(ratings, movies, on='movieId')

ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')



FileNotFoundError: [Errno 2] No such file or directory: '../머신러닝/data/ml-latest-small/movies.csv'

In [4]:
p, q = matrix_factorization(ratings_matrix.values, k=50, steps=200, learning_rate=0.01, r_lambda=0.01)

pred_matrix = np.dot(p, q.T)

step: 0 rmse: 2.9023619751336867
step: 10 rmse: 0.7335768591017927
step: 20 rmse: 0.5115539026853442
step: 30 rmse: 0.37261628282537446
step: 40 rmse: 0.2960818299181014
step: 50 rmse: 0.2520353192341642
step: 60 rmse: 0.22487503275269854
step: 70 rmse: 0.20685455302331537
step: 80 rmse: 0.19413418783028685
step: 90 rmse: 0.18470082002720403
step: 100 rmse: 0.17742927527209104
step: 110 rmse: 0.1716522696470749
step: 120 rmse: 0.1669518194687172
step: 130 rmse: 0.16305292191997542
step: 140 rmse: 0.15976691929679643
step: 150 rmse: 0.1569598699945732
step: 160 rmse: 0.15453398186715428
step: 170 rmse: 0.15241618551077643
step: 180 rmse: 0.15055080739628307
step: 190 rmse: 0.1488947091323209


In [5]:
!pip install scikit-surprise

In [24]:
import surprise
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [25]:
data  = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

In [26]:
algo = SVD(random_state = 0)
algo.fit(trainset)

In [27]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), 'size : ', len(predictions))
print('')
predictions[:5]

prediction type :  <class 'list'> size :  25000



[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [28]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [29]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [30]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

In [10]:
ratings = pd.read_csv('../8.머신러닝/data/ml-latest-small/ratings.csv')
movies = pd.read_csv('../8.머신러닝/data/ml-latest-small/movies.csv')

ratings.to_csv('../머신러닝/data/ml-latest-small/ratings_noh.csv', index = False, header = False)

In [13]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [32]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('../머신러닝/data/ml-latest-small/ratings_noh.csv', reader = reader)

In [33]:
data

In [34]:
trainset, testset = train_test_split(data, test_size =.25, random_state = 0)

algp = SVD(n_factors = 50, random_state = 0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8692


0.8691518972016722

In [38]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('../머신러닝/data/ml-latest-small/ratings.csv')

reader = Reader(rating_scale = (0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state = 0)
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8844  0.8751  0.8753  0.8685  0.8755  0.8758  0.0050  
MAE (testset)     0.6790  0.6707  0.6706  0.6690  0.6735  0.6726  0.0035  
Fit time          0.96    0.98    0.98    0.94    0.95    0.96    0.02    
Test time         0.13    0.25    0.13    0.14    0.14    0.16    0.05    


{'test_rmse': array([0.88437531, 0.87506717, 0.87531569, 0.86853258, 0.8755349 ]),
 'test_mae': array([0.67904406, 0.67067287, 0.6706233 , 0.66900812, 0.67352945]),
 'fit_time': (0.9634461402893066,
  0.9814448356628418,
  0.9754343032836914,
  0.9424624443054199,
  0.9474682807922363),
 'test_time': (0.13192486763000488,
  0.24785399436950684,
  0.134932279586792,
  0.13592290878295898,
  0.1359245777130127)}

In [40]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))

data_folds =  DatasetAutoFolds(ratings_file = '../머신러닝/data/ml-latest-small/ratings_noh.csv', reader = reader)

trainset = data_folds.build_full_trainset()

In [43]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)

algo.fit(trainset)

In [46]:
movies = pd.read_csv('../머신러닝/data/ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [49]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [56]:
def get_unseen_surprise(ratings, movies, userId) :
    
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    
    print('평점 매긴 영화수 : ', len(seen_movies), '추천대상 영화수 : ', len(unseen_movies),
         '전체 영화수 : ', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수 :  46 추천대상 영화수 :  9696 전체 영화수 :  9742


In [59]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10) :
    
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    
    def sortkey_est(pred) :
        
        return pred.est
    
    predictions.sort(key = sortkey_est, reverse = True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)
print('#### Top-10 추천 영화 리스트 ####')

for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화수 :  46 추천대상 영화수 :  9696 전체 영화수 :  9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
